In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
import warnings

sns.set()
rcParams['figure.figsize'] = (20,10)
pd.options.display.max_columns = None
warnings.filterwarnings('ignore')
from pprint import pprint
import json

from transformers import AutoTokenizer, AutoModel, AutoConfig, pipeline
from datasets import load_dataset, Dataset, concatenate_datasets

In [3]:
tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-3b")

### CNN_DAILY

In [ ]:
cnn = Dataset.from_json('../../data/translated/cnn_dailymail_30k_samples_len_100_1200_words_translated.json')

In [ ]:
input_template="Cuộc trò chuyện giữa con người và trợ lý AI.\n[|Con người|] Tóm tắt ngắn gọn đoạn văn bản sau đây:\n{article_translated}\n[|AI|] "
def mapper(x):
    return {
        'input': input_template.format(article_translated=x['article_translated']),
        'output': 'Đoạn văn bản đã được tóm tắt:\n' + x['highlights_translated'] + '\n[|Con người|]'
    }

temp_cnn = cnn.shuffle().select(range(15000)).map(mapper,batched=False, remove_columns=['article_translated','highlights_translated'])

In [ ]:
print(temp_cnn[1]['input'] + temp_cnn[1]['output'])

In [ ]:
length = temp_cnn.shuffle().select(range(1000)).map(lambda x: {'len':len(tokenizer.tokenize(x['input'] + x['output']))})
length.to_pandas().hist(bins=100)

In [ ]:
temp_cnn[0]

In [ ]:
temp_cnn.to_json('../../data/training_3_8/cnn_15k_samples.jsonl', orient='records', lines=True)

### DIALOG SUMMARY

In [ ]:
dialog = Dataset.from_json('../../data/translated/dialogsum_10k_samples_len_50_words_translated.json')

In [ ]:
dialog

In [ ]:
input_template="Tóm tắt ngắn gọn đoạn hội thoại sau đây:\n{dialogue_translated}\nĐoạn hội thoại đã được tóm tắt:\n"
def mapper(x):
    return {
        'input': input_template.format(dialogue_translated=x['dialogue_translated']),
        'output': x['summary_translated']
    }

temp_dialog = dialog.shuffle().select(range(10000)).map(mapper,batched=False, remove_columns=['dialogue_translated','summary_translated'])

In [ ]:
print(temp_dialog[1]['input'] + temp_dialog[1]['output'])

In [ ]:
temp_dialog[0]

In [ ]:
temp_dialog.to_json('../../data/training_3_8/dialogsum_10k_samples.jsonl', orient='records', lines=True)

### FAQS

In [ ]:
faqs = Dataset.from_json('../../data/training_31_7/faq_chat_10k.jsonl')

In [ ]:
length = faqs.map(lambda x: {'length':len(tokenizer.tokenize(x['input']))})

In [ ]:
pd.length['length']

In [ ]:
print(faqs.shuffle()[0]['input'])

In [ ]:
text = 'Cuộc trò chuyện giữa con người và trợ lý AI.\n'
text2 = ' Nếu không đủ thông tin để trả lời thì trả lời: Tôi không biết.'
text3 = 'Điều luật liên quan: '
text4 = 'Chỉ được trả lời dựa trên điều luật được cung cấp'
def mapper(x):
    return {
        'input': text,
        'output': x['input'].removeprefix(text).replace(text2,'').replace(text3, 'Điều luật liên quan:\n').replace(text4, 'Chỉ được trả lời dựa trên thông tin nằm trong điều luật được cung cấp') + '\n[|Con người|]',
    }

temp_faqs = faqs.map(mapper, batched=False)

In [ ]:
temp = temp_faqs.shuffle()[0]
temp

In [ ]:
print(temp['input'] + temp['output'])

In [ ]:
temp_faqs.to_json('../../data/training_3_8/faqs.jsonl', orient='records', lines=True)

### SHARE_GPT

In [58]:
share = pd.read_json('/Users/phamhoang1408/Desktop/Phase 2 Viettel/main_repo/data/original/share_gpt_no_code_conversations_40k.json', orient='records', lines=True)
share.columns = ['input']
share.dropna(inplace=True)
share.reset_index(drop=True, inplace=True)
share = Dataset.from_pandas(share)

In [ ]:
share = share.filter(lambda x: len(tokenizer.tokenize(x["input"])) < 2000)

In [ ]:
share.shuffle().select(range(1000)).map(lambda x: {'len':len(tokenizer.tokenize(x['input']))}).to_pandas().hist(bins=100)

In [59]:
# text = 'Cuộc trò chuyện giữa con người và trợ lý AI.\n'
text = 'The conversation between human and AI assistant.\n'
def mapper(x):
    return {
        'input': text,
        'output': x['input'].removeprefix(text) + '[|Human|]'
    }
share = share.map(mapper, batched=False)

Map:   0%|          | 0/38419 [00:00<?, ? examples/s]

In [51]:
share

Dataset({
    features: ['input', 'output'],
    num_rows: 38419
})

In [62]:
temp = share.shuffle()[0]
print(temp['input'] + temp['output'])

The conversation between human and AI assistant.
[|AI|] The immune response is the body's response to the presence of a foreign substance, such as a pathogen or toxin. The immune response involves the activation of various cells, tissues, and organs of the immune system to recognize and eliminate the foreign substance.

Here are some key features of the immune response:

1. Specificity: The immune response is specific to the foreign substance that is present, meaning that it is able to recognize and attack a particular pathogen or toxin.
2. Diversity: The immune system has a diverse range of cells and proteins that can recognize and attack a wide range of foreign substances.
3. Memory: The immune system has the ability to "remember" past infections, allowing it to mount a more rapid and effective response to future encounters with the same pathogen.
4. Cooperation: The immune response involves the coordination and cooperation of various cells and tissues, such as T cells, B cells, and 

In [47]:
share

Dataset({
    features: ['input', 'output'],
    num_rows: 38419
})

In [63]:
share.to_json('../../data/training_english/share_gpt_38k.jsonl', orient='records', lines=True)

Creating json from Arrow format:   0%|          | 0/39 [00:00<?, ?ba/s]

294659991

### QUORA

In [ ]:
quora = Dataset.from_json('../../data/translated/quora_chat_data_translated.json')

In [ ]:
print(quora.shuffle()[0]['prompt'])

In [ ]:
quora.shuffle().select(range(1000)).map(lambda x: {'len':len(tokenizer.tokenize(x['prompt']))}).to_pandas().hist(bins=100)

In [ ]:
temp_quora = quora.shuffle().select(range(20000))
temp_quora

In [ ]:
text = 'Cuộc trò chuyện giữa con người và trợ lý AI.\n'
def mapper(x):
    return {
        'input': text,
        'output': x['prompt'].removeprefix(text) + '\n[|Con người|]',
    }

temp_quora = temp_quora.map(mapper, batched=False, remove_columns=['prompt'])

In [ ]:
temp_quora.to_json('../../data/training_3_8/quora_20k_samples.jsonl', orient='records', lines=True)

### OASST

In [4]:
# en_oasst = Dataset.from_json('../../data/translated/en_oasst_translated.json')
# en_oasst = en_oasst.rename_column('text_translated','text')
# vi_oasst = Dataset.from_json('../../data/original/oasst/vi_oasst.json')

ds = Dataset.from_json('/Users/phamhoang1408/Desktop/Phase 2 Viettel/main_repo/data/original/oasst/en_oasst.json')

Found cached dataset json (/Users/phamhoang1408/.cache/huggingface/datasets/json/default-c507d95be8d43b3a/0.0.0)


In [27]:
ds[0]

{'text': 'The conversation between human and AI assistant.\n[|Human|] Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.\n[|AI|] "Monopsony" refers to a market structure where there is only one buyer for a particular good or service. In economics, this term is particularly relevant in the labor market, where a monopsony employer has significant power over the wages and working conditions of their employees. The presence of a monopsony can result in lower wages and reduced employment opportunities for workers, as the employer has little incentive to increase wages or provide better working conditions.\n\nRecent research has identified potential monopsonies in industries such as retail and fast food, where a few large companies control a significant portion of the market (Bivens & Mishel, 2013). In these industries, workers often face low wages, 

In [14]:
# text = 'Cuộc trò chuyện giữa con người và trợ lý AI.\n'
text = 'The conversation between human and AI assistant.\n'
def mapper(x):
    return {
        'input': text,
        'output': x['text'].removeprefix(text),
    }

temp_oasst = ds.map(mapper, batched=False, remove_columns=['text'])

Map:   0%|          | 0/21164 [00:00<?, ? examples/s]

In [15]:
temp_oasst

Dataset({
    features: ['input', 'output'],
    num_rows: 21164
})

In [69]:
temp = temp_oasst.shuffle()[0]
print(temp['input'] + temp['output'])

The conversation between human and AI assistant.
[|Human|] Explain the rules of field hockey to a novice.
[|AI|] Field hockey is a team sport played on a rectangular field, typically measuring 100 yards long and 60 yards wide. Each team consists of 11 players, including a goalkeeper. The objective of the game is to score more goals than the opposing team by hitting a small, hard ball into the opponent's goal using a hockey stick.

Here are the basic rules of field hockey:

1. Starting the game: The game starts with a coin toss to determine which team will take the first pass. The team that wins the toss starts with the ball in the center of the field.

2. Game duration: A game is typically divided into two halves of 35 minutes each, with a 10-minute halftime break in between.

3. Scoring: A goal is scored when the ball completely crosses the opponent's goal line between the goalposts and under the crossbar.

4. Equipment: Players wear shin guards, mouthguards, and usually cleats. The g

In [ ]:
temp_oasst.to_json('../../data/training_3_8/oasst_21k_samples.jsonl', orient='records', lines=True)

In [ ]:
oasst.shuffle().select(range(1000)).map(lambda x: {'len':len(tokenizer.tokenize(x['text']))}).to_pandas().hist(bins=100)

In [ ]:
print(temp_oasst.shuffle()[0])

In [70]:
temp_oasst.to_json('../../data/training_english/oasst_20k_samples.jsonl', orient='records', lines=True)

Creating json from Arrow format:   0%|          | 0/22 [00:00<?, ?ba/s]

35941652

### ALPACA CHAT

In [ ]:
alpaca = Dataset.from_json('../../data/training_31_7/alpaca_chat_15k.jsonl')
alpaca

In [ ]:
alpaca

In [ ]:
text = 'Cuộc trò chuyện giữa con người và trợ lý AI.\n'
def mapper(x):
    return {
        'input': text,
        'output': x['input'].removeprefix(text)
    }
temp_alpaca = alpaca.map(mapper, batched=False, remove_columns=['input'])

In [ ]:
temp_alpaca[0]

In [ ]:
temp_alpaca.to_json('../../data/training_3_8/alpaca_chat_15k_samples.jsonl', orient='records', lines=True)

### GPT4 INSTRUCT

In [93]:
gpt4 = Dataset.from_json('/Users/phamhoang1408/Desktop/Phase 2 Viettel/main_repo/data/original/gpt4_instruct_similarity_0_9.json')
gpt4

Found cached dataset json (/Users/phamhoang1408/.cache/huggingface/datasets/json/default-1ec4cd6bcffd1298/0.0.0)


Dataset({
    features: ['instruction', 'input', 'response'],
    num_rows: 17873
})

In [72]:
gpt4

Dataset({
    features: ['instruction', 'input', 'response'],
    num_rows: 17873
})

In [94]:
template1 = "[|Human|] {instruction}\n{input}\n[|AI|] "
template2 = "[|Human|] {instruction}\n[|AI|] "

def mapper(x):
    if x['input'] != '' and x['input'] is not None:
        template = template1.format(
            instruction=x['instruction'], 
            input=x['input']
        )
    else:
        template = template2.format(instruction=x['instruction'])
    return {
        'input': template,
        'output': x['response']
    }

temp_gpt4 = gpt4.map(mapper,batched=False, remove_columns=gpt4.column_names)

Map:   0%|          | 0/17873 [00:00<?, ? examples/s]

In [95]:
temp_gpt4

Dataset({
    features: ['input', 'output'],
    num_rows: 17873
})

In [100]:
a = temp_gpt4.shuffle()[0]
print(a['input']+a['output'])

[|Human|] Rewrite the following paragraph to be more concise.
One of the key aspects of nanotechnology, which involves the application of engineering at the molecular and atomic level, is that it has the potential to provide significant advancements in a variety of fields. These may include areas such as energy production, medical applications, environmental management, and material science. Due to the inherently smaller scale upon which nanotechnology operates, researchers and engineers have the ability to manipulate and control materials at the atomic level. This can lead to more efficient, effective, and powerful products and technologies being developed.
[|AI|] Nanotechnology, the manipulation of materials at the molecular and atomic level, has the potential to revolutionize fields like energy production, medicine, environmental management, and material science by creating more efficient and powerful products.


In [101]:
temp_gpt4.to_json('../../data/training_english/gpt4_instruct.jsonl', orient='records', lines=True)

Creating json from Arrow format:   0%|          | 0/18 [00:00<?, ?ba/s]

11710929

### DOLLY WITH CONTEXT

In [149]:
dolly = Dataset.from_json('/Users/phamhoang1408/Desktop/Phase 2 Viettel/main_repo/data/original/dolly_with_context.jsonl')

Found cached dataset json (/Users/phamhoang1408/.cache/huggingface/datasets/json/default-88170a2f4a3af818/0.0.0)


In [150]:
dolly.shuffle()[0]

{'instruction': 'What is handscroll used for',
 'context': 'The handscroll is a long, narrow, horizontal scroll format in East Asia used for calligraphy or paintings. A handscroll usually measures up to several meters in length and around 25–40 cm in height. Handscrolls are generally viewed starting from the right end. This kind of scroll is intended to be read or viewed flat on a table, in sections. The format thus allows for the depiction of a continuous narrative or journey. The traditional alternative format in East Asian paintings is the vertical hanging scroll, which is rarely as long.',
 'response': 'The handscroll is a long, narrow, horizontal scroll format in East Asia used for calligraphy or paintings.',
 'category': 'closed_qa'}

In [151]:
# template1 = "The conversation between human and AI assistant.\n[|Human|] Trả lời câu hỏi dựa vào đoạn văn bản dưới đây. Chỉ được trả lời dựa trên thông tin nằm trong văn bản được cung cấp.\nCâu hỏi: {instruction}\nĐoạn văn bản:\n{context}\n[|AI|] "
template1 = "[|Human|] Answer the question given the following context.\nQuestion: {instruction}\Context:\n{context}\n[|AI|] "

def mapper(x):
    ip = template1.format(
        instruction=x['instruction'], 
        context=x['context']
    )
    return {
        'input': ip,
        'output': x['response']
    }

temp_dolly = dolly.map(mapper,batched=False, remove_columns=dolly.column_names)

Loading cached processed dataset at /Users/phamhoang1408/.cache/huggingface/datasets/json/default-88170a2f4a3af818/0.0.0/cache-cc2e3641bc588444.arrow


In [152]:
df = temp_dolly.to_pandas()
df = df.sample(8000, random_state=42, replace=True)
df = df.reset_index(drop=True)
temp_dolly = Dataset.from_pandas(df)

In [91]:
temp = temp_dolly.shuffle()[0]
print(temp['input'] + temp['output'])

[|Human|] Answer the question given the following context.
Question: Extract the different storage configurations available to the iPhone 14 that are mentioned in the text. Put them in a comma separated list.\Context:
iPhone 14 and 14 Plus are available in three internal storage configurations: 128, 256, and 512 GB. It has 6 GB of RAM, an increase over the iPhone 13 and 13 mini model's 4 GB of RAM. The iPhone 14 and 14 Plus have the same IP68 rating for dust and water resistance as their predecessors.
[|AI|] 128 GB, 256 GB, 512 GB


Cuộc trò chuyện giữa con người và trợ lý AI.
[|Con người|] Trả lời câu hỏi pháp luật dựa vào những điều luật liên quan dưới đây. Chỉ được trả lời dựa trên thông tin nằm trong điều luật được cung cấp.
Câu hỏi: Phạm tội trong trạng thái tinh thần bị kích động mạnh bị xử lý thế nào?
Điều luật liên quan:
Điều 125. Tội giết người trong trạng thái tinh thần bị kích động mạnh 1. Người nào giết người trong trạng thái tinh thần bị kích động mạnh do hành vi trái pháp luật nghiêm trọng của nạn nhân đối với người đó hoặc đối với người thân thích của người đó, thì bị phạt tù từ 06 tháng đến 03 năm. 2. Phạm tội đối với 02 người trở lên, thì bị phạt tù từ 03 năm đến 07 năm.
[|AI|] Giết người trong trạng thái tinh thần bị kích động mạnh Tội giết người trong trạng thái tinh thần bị kích động mạnh được quy định tại Điều 125 Bộ luật Hình sự 2015 sửa đổi, bổ sung 2017 với mức hình phạt như sau: Khung 1: Người nào giết người trong trạng thái tinh thần bị kích động mạnh do hành vi trái pháp luật nghiêm trọng của nạn nhân đối với người đó hoặc đối với người thân thích của người đó, thì bị phạt tù từ 06 tháng đến 03 năm. Khung 2: Phạm tội đối với 02 người trở lên, thì bị phạt tù từ 03 năm đến 07 năm.
[|Con người|]

In [153]:
temp_dolly.to_json('../../data/training_english/dolly_4k_samples.jsonl', orient='records', lines=True)

Creating json from Arrow format:   0%|          | 0/8 [00:00<?, ?ba/s]

12791456

### Load all

In [154]:
from glob import glob
def load_dataset(folder_path, dataset_size=None):
    data = []
    for path in glob(folder_path + "/*.jsonl"):
        ds = Dataset.from_json(path)
        for check in ['input', 'output']:
            if check not in ds.column_names:
                raise ValueError("Dataset must have input, output columns")
        # take only the input and output columns
        ds = ds.select_columns(['input', 'output'])
        data.append(ds)
    if dataset_size is None:
        return concatenate_datasets(data, axis=0).shuffle(seed=42)
    return (
        concatenate_datasets(data, axis=0).shuffle(seed=42).select(range(dataset_size))
    )

def clean(ds):
    def mapper(x):
        for column in ds.column_names:
            x[column] = x[column].replace('[|Human|]','[|Con người|]').replace('\n[|Con người|]\n[|Con người|]', '\n[|Con người|]')
        return x
    def filter(x):
        for column in ds.column_names:
            if x[column] == '' or x[column] is None:
                return False
        return True

    new_ds = ds.map(mapper,batched=False)
    new_ds = new_ds.filter(filter)
    return new_ds        

In [155]:
ds = load_dataset('../../data/training_english')

Found cached dataset json (/Users/phamhoang1408/.cache/huggingface/datasets/json/default-154e9ac7b45b6cea/0.0.0)


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Found cached dataset json (/Users/phamhoang1408/.cache/huggingface/datasets/json/default-f2def6352b4e3016/0.0.0)
Found cached dataset json (/Users/phamhoang1408/.cache/huggingface/datasets/json/default-e8f787b4afeee828/0.0.0)


Dataset json downloaded and prepared to /Users/phamhoang1408/.cache/huggingface/datasets/json/default-0721c5b65b2c181d/0.0.0. Subsequent calls will reuse this data.


In [156]:
ds

Dataset({
    features: ['input', 'output'],
    num_rows: 85456
})

In [216]:
a = ds.shuffle()[0]
print(a['input'])
print(a['output'])

The conversation between human and AI assistant.

[|Human|] What is the typical wattage of bulb in a lightbox?
[|AI|] The amount of electricity that a light bulb uses depends on its size and type. Traditional incandescent bulbs usually consume between 25 to 100 watts, while LED bulbs use between 2 to 18 watts. Most light bulbs draw around 110 volts and less than 1 amp.

However, the biggest factor that affects the amount of electricity a light bulb uses is how often it is used. In the United States, the average time that lights are on per day is about 2 hours. For example, using an average 60 watt incandescent bulb:

Using the light bulb for 1 hour per day results in 0.42 kilowatt-hours (kWh) of electricity per week, 1.83 kWh per month, and 21.9 kWh per year.
Using the light bulb for 2 hours per day results in 0.84 kWh per week, 3.65 kWh per month, and 43.8 kWh per year.
On the other hand, using the light bulb for 3 hours per day results in 1.26 kWh of electricity per week, 5.48 kWh pe

In [162]:
path_to_save = '../../data/english_training/all_11_8.jsonl'
ds.to_json(path_to_save, orient='records', lines=True)

Creating json from Arrow format:   0%|          | 0/86 [00:00<?, ?ba/s]

355104028

In [165]:
ds

Dataset({
    features: ['input', 'output'],
    num_rows: 85456
})

In [ ]:
ds.shuffle().select(range(1000)).map(lambda x: {'len':len(tokenizer.tokenize(x['input']+x['output']))}).to_pandas().hist(bins=100)

In [ ]:
df = ds.to_pandas()

In [ ]:
df_length = df.apply(lambda x: len(tokenizer.tokenize(x['input']+x['output'])), axis=1)

In [ ]:
temp_df = df[df_length <= 512].reset_index(drop=True)
temp_df.shape

In [ ]:
final_df = temp_df.sample(60000).reset_index(drop=True)

In [ ]:
ds = Dataset.from_pandas(final_df)

In [ ]:
ds.to_json('../../data/temp_training/final_dataset.jsonl', orient='records', lines=True)

In [ ]:
ds = Dataset.from_json('/Users/phamhoang1408/Desktop/Phase 2 Viettel/main_repo/data/original/formatted_toolformer-similarity-0.8-dataset.json')

In [ ]:
ds.shuffle()[0]

In [4]:
wizard = load_dataset('WizardLM/WizardLM_evol_instruct_V2_196k')['train']

Found cached dataset json (/Users/phamhoang1408/.cache/huggingface/datasets/WizardLM___json/WizardLM--WizardLM_evol_instruct_V2_196k-ce2aea5a7424f8ea/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
wizard.to_json('../../../data/original/wizard_full.jsonl', orient='records', lines=True)

Creating json from Arrow format:   0%|          | 0/143 [00:00<?, ?ba/s]

351972540